# Lesson 2: Retrieving a Single Record by ID

# Retrieving a Single Record by ID

Welcome to the section on **retrieving a single record by its ID** using Django ORM. Previously, we explored how to retrieve all records from a database. Now, we'll zoom in and focus on fetching a specific record. This is an essential skill for many real-world applications where you need detailed information about a particular item.

## What You'll Learn
In this lesson, you'll learn how to retrieve a single record from the database using its ID. This is a common scenario when the user needs to view or interact with detailed data about a single item.

We'll work with the `Todo` model that we used previously:

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

## Retrieving a Record by ID

To retrieve a specific `Todo` item by ID, we'll use the `get()` method of the Django ORM. This method allows us to fetch a single record based on a specific field value, such as the primary key (ID).

```python
from django.http import JsonResponse
from django.shortcuts import get_object_or_404
from .models import Todo

def get_todo(request, id):
    try:
        todo = Todo.objects.get(id=id)
        return JsonResponse({'id': todo.id, 'task': todo.task})
    except Todo.DoesNotExist:
        return JsonResponse({'message': 'Todo not found'}, status=404)
```

This function checks if a `Todo` with the given ID exists. If it does, it retrieves the `Todo` object and returns its details as a JSON response. If the record doesn't exist, a 404 error is returned.

### Using `get_object_or_404()`
Django also provides a shortcut method, `get_object_or_404()`, that simplifies this process. It retrieves the object if it exists, or raises a 404 error if not found.

```python
def get_todo(request, id):
    todo = get_object_or_404(Todo, id=id)
    return JsonResponse({'id': todo.id, 'task': todo.task})
```

In this case, if the element with the given ID is not found, a 404 error will be raised automatically, and a 404 page will be displayed.

## Why It Matters

Learning how to retrieve a single record by its ID is vital for building interactive and user-friendly applications. Consider scenarios like:

- 🛒 Viewing the details of a specific product on an e-commerce site.
- 📰 Displaying a single blog post on a news website.
- ✅ Showing detailed information about a task in a to-do list application.

Being able to fetch and display precise data improves the user experience by allowing users to see the information they need quickly and efficiently. It's a fundamental skill that will make your Django applications more responsive and dynamic.

## Ready to Get Started?

Let's move on to the practice section and apply the concepts we've discussed. With hands-on practice, you'll build your confidence in retrieving single records by ID. Happy coding! 🎉

## Retrieving Single Record by ID

### Running the Code to Retrieve a Single Record by ID

In this lesson, we're focusing on preloading some tasks into the database and then retrieving them by their IDs. The code sets up the logic to preload some `Todo` tasks and provides a way to retrieve those tasks via their unique ID.

### Explanation of the Code

1. **Preload Tasks:**
   - The `preload()` function checks if tasks already exist in the database and creates them if they don't. This ensures we have some data to work with before fetching specific records.
   
   ```python
   @csrf_exempt
   def preload(request):
       tasks = [('Do the laundry', False), ('Buy groceries', False), ('Call mom', True)]
       for task, completed in tasks:
           if not Todo.objects.filter(task=task).exists():
               Todo.objects.create(task=task, completed=completed)
       return JsonResponse({'message': 'Preloaded successfully'})
   ```

2. **Retrieve a Single Task by ID:**
   - The `get_todo()` function retrieves a specific `Todo` task by its `ID`. If the task exists, it returns the task details in JSON format; otherwise, it returns a "Todo not found" message with a 404 status code.
   
   ```python
   def get_todo(request, id):
       try:
           todo = Todo.objects.get(id=id)
           return JsonResponse({'id': todo.id, 'task': todo.task})
       except Todo.DoesNotExist:
           return JsonResponse({'message': 'Todo not found'}, status=404)
   ```

3. **Model Definition:**
   - The `Todo` model defines the structure of each task, containing a `task` description and a `completed` status (boolean). This model will be used to store and retrieve data from the database.
   
   ```python
   class Todo(models.Model):
       task = models.CharField(max_length=200)
       completed = models.BooleanField(default=False)

       def __str__(self):
           return self.task
   ```

4. **URL Configuration:**
   - The `urlpatterns` define the paths to access the `preload()` and `get_todo()` views. The `/preload/` path is used to preload the tasks, and `/todos/<int:id>/` is used to fetch a specific task by its `ID`.
   
   ```python
   from django.urls import path
   from myapp import views

   urlpatterns = [
       path('preload/', views.preload, name='preload'),
       path('todos/<int:id>/', views.get_todo, name='get_todo'),
   ]
   ```

5. **Client Script to Preload and Fetch Tasks:**
   - The client-side script sends a request to preload the tasks, then retrieves the task with `ID=1`. It also sends a request for an invalid ID (`444`) to demonstrate error handling.

   ```python
   import requests

   URL = 'http://127.0.0.1:3000'

   # Preload the database
   response = requests.get(URL + '/preload/')
   print(response.json())

   # Retrieve a specific todo by ID
   todo_id = 1
   get_response = requests.get(URL + f'/todos/{todo_id}/')
   print(get_response.json())

   # Invalid request for a non-existing todo
   response = requests.get(URL + '/todos/444/')
   print(response.json())
   ```

### Expected Output

- **Preload Response:**
  When you send a request to `/preload/`, the server will create the tasks and return:
  ```json
  {"message": "Preloaded successfully"}
  ```

- **Fetching a Valid Todo (ID=1):**
  When you request `/todos/1/`, assuming the first task is loaded correctly, the response will be:
  ```json
  {"id": 1, "task": "Do the laundry"}
  ```

- **Fetching an Invalid Todo (ID=444):**
  When you request `/todos/444/`, the server will return a "not found" message:
  ```json
  {"message": "Todo not found"}
  ```

### Conclusion

By running the code, you can observe how preloading data into the database and retrieving it by ID works in Django. This is a useful pattern for many applications that require retrieving specific records based on user requests or interactions.

## Retrieve Task by Name

### Modifying the Function to Retrieve Todo by Task Name

Let's update the existing code so that instead of retrieving a Todo task by its ID, we will retrieve it using the task name. This change will allow us to query based on the `task` field, which is a string, rather than the numeric `ID`. Additionally, we'll update the URL pattern to reflect the change.

### Step-by-Step Modifications

1. **Update the `get_todo()` function to retrieve by task name:**

   Instead of using the task's `ID`, we will now use the `task` field to search for the specific Todo.

   ```python
   from django.http import JsonResponse
   from django.shortcuts import get_object_or_404
   from .models import Todo
   import json
   from django.views.decorators.csrf import csrf_exempt

   @csrf_exempt
   def preload(request):
       tasks = [('Do the laundry', False), ('Buy groceries', False), ('Call mom', True)]
       for task, completed in tasks:
           if not Todo.objects.filter(task=task).exists():
               Todo.objects.create(task=task, completed=completed)
       return JsonResponse({'message': 'Preloaded successfully'})

   # Change this function to retrieve a task by its name instead of its ID
   def get_todo_by_name(request, task_name):
       try:
           todo = Todo.objects.get(task=task_name)
           return JsonResponse({'id': todo.id, 'task': todo.task, 'completed': todo.completed})
       except Todo.DoesNotExist:
           return JsonResponse({'error': 'Task not found'}, status=404)
   ```

   - This code now retrieves a `Todo` object by its `task` name.
   - The query has been changed to `Todo.objects.get(task=task_name)` to fetch the record based on the name of the task.

2. **Update the URL pattern to use task names:**

   Modify the `urls.py` file to accept the `task_name` parameter in the URL instead of the numeric ID.

   ```python
   from django.urls import path
   from myapp import views

   urlpatterns = [
       path('preload/', views.preload, name='preload'),
       # Update this URL pattern to use the task name instead of the task ID
       path('todos/<str:task_name>/', views.get_todo_by_name, name='get_todo_by_name'),
   ]
   ```

   - The pattern `<str:task_name>` is used to match a string value for the `task_name` in the URL.

3. **Client Script to Test the Changes:**

   In the client script, we'll update the request to use the task name when querying the `Todo` object.

   ```python
   import requests

   URL = 'http://127.0.0.1:3000'

   # Preload the database
   response = requests.get(URL + '/preload/')
   print(response.json())

   # Retrieve a specific todo by task name
   task_name = 'Do the laundry'
   get_response = requests.get(URL + f'/todos/{task_name}/')
   print(get_response.json())

   # Invalid request for a non-existing task
   response = requests.get(URL + '/todos/Invalid Task/')
   print(response.json())
   ```

   - The script now sends a request to the `/todos/<task_name>/` endpoint to retrieve tasks by their name.
   - In the invalid request, we use a task name that doesn’t exist to simulate an error.

### Expected Output

1. **Preload Response:**
   When the tasks are preloaded, the response should indicate success:
   ```json
   {"message": "Preloaded successfully"}
   ```

2. **Fetching a Valid Todo by Task Name (`"Do the laundry"`):**
   When querying for an existing task, the response will be:
   ```json
   {"id": 1, "task": "Do the laundry", "completed": false}
   ```

3. **Fetching an Invalid Task (`"Invalid Task"`):**
   If the task name doesn't exist, the response will be:
   ```json
   {"error": "Task not found"}
   ```

### Conclusion

By modifying the query to retrieve a `Todo` task based on its name, we've demonstrated how to work with string-based parameters in Django. This pattern is useful when querying objects by fields other than their primary key, such as names or other unique identifiers.

## Retrieve Todo by ID or Name

To modify the `get_todos_by_status` view so that it filters the `Todo` objects based on the `completed_status` and checks whether the word "Call" is in the `task` field, we can leverage Django's `Q` object for complex queries. We'll also need to use the `icontains` filter to perform a case-insensitive search for "Call" in the `task` field.

Let's go step by step to implement this:

### Step-by-Step Implementation

1. **Update the `get_todos_by_status` View:**

   The updated view will filter based on both the `completed_status` and the presence of "Call" in the `task` field.

   ```python
   from django.http import JsonResponse
   from django.db.models import Q
   from .models import Todo
   import json
   from django.views.decorators.csrf import csrf_exempt

   @csrf_exempt
   def preload(request):
       tasks = [('Do the laundry', False), ('Buy groceries', False), ('Call mom', True), ('Call dad', False)]
       for task, completed in tasks:
           if not Todo.objects.filter(task=task).exists():
               Todo.objects.create(task=task, completed=completed)
       return JsonResponse({'message': 'Preloaded successfully'})

   def get_todos_by_status(request, completed_status):
       # Convert the completed_status parameter to a boolean
       completed_status_bool = completed_status.lower() == 'true'
       
       # Use Q objects to filter tasks that are completed/incompleted and contain 'Call' in the task name
       todos = Todo.objects.filter(
           Q(completed=completed_status_bool) & Q(task__icontains='Call')
       ).values()

       return JsonResponse(list(todos), safe=False)
   ```

   **Explanation:**
   - The `Q` object allows us to perform complex queries. In this case, it is used to combine the two conditions: 
     1. `completed=completed_status_bool` checks whether the task is completed or not based on the URL parameter.
     2. `task__icontains='Call'` filters tasks that have the word "Call" in the task name, ignoring case.
   - The result is returned as a list of filtered todos in JSON format.

2. **Update the URL Patterns:**

   We will keep the URL pattern the same, as it is already set up to accept the `completed_status` parameter.

   ```python
   from django.urls import path
   from myapp import views

   urlpatterns = [
       path('preload/', views.preload, name='preload'),
       path('todos/status/<str:completed_status>/', views.get_todos_by_status, name='get_todos_by_status'),
   ]
   ```

3. **Update the Client Script to Test the Changes:**

   In the client script, we will test both `True` and `False` completed statuses and see if the correct tasks are retrieved.

   ```python
   import requests

   URL = 'http://127.0.0.1:3000'

   # Preload the database
   response = requests.get(URL + '/preload/')
   print(response.json())

   # Retrieve todos with completed status 'True' and that have 'Call' in the task name
   completed_status = 'True'
   get_response = requests.get(URL + f'/todos/status/{completed_status}/')
   print(get_response.json())  # Expected to return tasks like 'Call mom'

   # Retrieve todos with completed status 'False' and that have 'Call' in the task name
   completed_status = 'False'
   get_response = requests.get(URL + f'/todos/status/{completed_status}/')
   print(get_response.json())  # Expected to return tasks like 'Call dad'
   ```

### Expected Output

1. **Preload Response:**
   After running the preload script, the response should indicate success:
   ```json
   {"message": "Preloaded successfully"}
   ```

2. **Fetching Todos with Completed Status `True` and Task Contains "Call":**
   If you query for tasks that are completed and have "Call" in their name, the result will include tasks like:
   ```json
   [{"id": 3, "task": "Call mom", "completed": true}]
   ```

3. **Fetching Todos with Completed Status `False` and Task Contains "Call":**
   Similarly, for tasks that are not completed but contain "Call", the response will be:
   ```json
   [{"id": 4, "task": "Call dad", "completed": false}]
   ```

### Conclusion

By modifying the `get_todos_by_status` view to use `Q` objects and `icontains`, we can now filter the `Todo` tasks based on both the `completed_status` and the presence of the word "Call" in the task name. This technique is useful for constructing complex queries and providing more refined search results.

## Retrieve Todo with Django ORM

To complete the `get_todo` view function in Django, you'll need to retrieve the `Todo` item by its ID, handle the case where the item doesn't exist, and return the appropriate JSON response.

Here's how you can implement it:

1. Use `get_object_or_404` to attempt to retrieve the `Todo` object by its ID. If the item is not found, return a custom JSON response indicating that the item doesn't exist.
2. If the `Todo` item is found, return a JSON response containing the details of the task.

Here is the updated `get_todo` function:

```python
from django.http import JsonResponse
from django.shortcuts import get_object_or_404
from .models import Todo
import json
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def preload(request):
    tasks = [('Do the laundry', False), ('Buy groceries', False), ('Call mom', True)]
    for task, completed in tasks:
        if not Todo.objects.filter(task=task).exists():
            Todo.objects.create(task=task, completed=completed)
    return JsonResponse({'message': 'Preloaded successfully'})

def get_todo(request, id):
    try:
        # Fetch the Todo item by ID or raise a 404 error if not found
        todo = get_object_or_404(Todo, id=id)
        # Return the Todo details as a JSON response
        return JsonResponse({
            'id': todo.id,
            'task': todo.task,
            'completed': todo.completed
        })
    except Todo.DoesNotExist:
        # If the Todo item does not exist, return a not found message
        return JsonResponse({'message': 'Todo item not found'}, status=404)
```

### Explanation:
1. **Fetching Todo item**: `get_object_or_404(Todo, id=id)` fetches the `Todo` item by its ID. If it doesn't exist, a 404 error is automatically raised.
2. **Returning Todo details**: If the item is found, we return a JSON response with its `id`, `task`, and `completed` fields.
3. **Handling missing items**: If the item is not found, a `JsonResponse` with a message `'Todo item not found'` and a `404` status code is returned.

### Test Cases:
- **Valid request**: Fetch a `Todo` with a valid ID, e.g., `GET /todos/1/`, and it should return the details of the task.
- **Invalid request**: Fetch a `Todo` with a non-existing ID, e.g., `GET /todos/444/`, and it should return `{"message": "Todo item not found"}` with a `404` status.

### URLs:
The URL patterns you provided remain the same, and this function will handle both the valid and invalid cases for retrieving a `Todo` item.

Let me know if you need further adjustments!

The error you’re encountering suggests that the response you're getting from the server isn't a valid JSON format, and `requests` is trying to decode it as JSON but failing.

### Possible Causes:
1. **The server didn't return a JSON response**: This can happen if the server encountered an error and returned an HTML error page (like a 500 or 404 HTML response) instead of JSON.
2. **CSRF token issue**: Since Django has CSRF protection enabled by default, and you're using `@csrf_exempt` for the `preload` view, it may not be responding as expected.
3. **Empty response**: The server might have sent an empty response, which leads to `json.decoder.JSONDecodeError` because there's nothing to decode.

### Debugging Steps:
1. **Check the Status Code**: Before trying to decode the response as JSON, check the HTTP status code to ensure the server responded successfully.
2. **Print the Response Text**: Log the raw response to see what exactly the server is returning.

Here's an updated version of your `send_request.py` to help debug:

```python
import requests

URL = 'http://127.0.0.1:3000'

# Preload the database
response = requests.get(URL + '/preload/')

# Check if the status code is not 200 (success)
if response.status_code != 200:
    print(f"Error: Received status code {response.status_code}")
    print("Response Text:", response.text)
else:
    try:
        print(response.json())
    except requests.exceptions.JSONDecodeError:
        print("Failed to decode JSON. Raw response:")
        print(response.text)

# Try fetching a specific todo
todo_id = 1
get_response = requests.get(URL + f'/todos/{todo_id}/')

if get_response.status_code != 200:
    print(f"Error: Received status code {get_response.status_code}")
    print("Response Text:", get_response.text)
else:
    try:
        print(get_response.json())
    except requests.exceptions.JSONDecodeError:
        print("Failed to decode JSON. Raw response:")
        print(get_response.text)
```

### Things to Look For:
- **Status Code**: If the server returns a status code like `500` or `404`, that means the issue is likely in the Django app.
- **Response Text**: If you see HTML content (like an error page) or an empty response, you'll have more insight into what’s wrong.

### Possible Fixes:
1. **Ensure Django Views Return JSON**: Make sure the `preload` and `get_todo` views always return a valid JSON response, even in case of errors.
   - For example, if the database has issues, you should handle that in the views and return an appropriate error message in JSON format.
   
2. **Ensure Django Server is Running Correctly**: Make sure the Django development server is running and accessible at the URL you're using (`http://127.0.0.1:3000`).

Let me know if this reveals any further information!

## Fetch Todos by Category and Tag

To implement the view that retrieves `Todo` items based on both the `category` and `tag` fields, we need to:

1. **Check if `category` and `tag` are provided**: If they are not provided in the query string, we should return an error message.
2. **Filter the `Todo` items**: Use Django’s `filter()` method to find all `Todo` items that match both the `category` and `tag`.
3. **Return the results in JSON format**: If matching `Todo` items are found, return their details as a JSON response.

Here’s the updated implementation:

### Updated `views.py`:

```python
from django.http import JsonResponse
from .models import Todo
import json
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def preload(request):
    tasks = [
        ('Do the laundry', False, 'Home', 'Urgent'),
        ('Buy groceries', False, 'Shopping', 'Weekly'),
        ('Call mom', True, 'Personal', 'Family')
    ]
    for task, completed, category, tag in tasks:
        if not Todo.objects.filter(task=task).exists():
            Todo.objects.create(task=task, completed=completed, category=category, tag=tag)
    return JsonResponse({'message': 'Preloaded successfully'})

def get_todos_by_category_and_tag(request):
    # Retrieve the 'category' and 'tag' parameters from the request
    category = request.GET.get('category')
    tag = request.GET.get('tag')
    
    # Check if both 'category' and 'tag' are provided
    if not category or not tag:
        return JsonResponse({'message': 'Category and/or Tag not provided'}, status=400)
    
    # Filter the Todo items that match both category and tag
    todos = Todo.objects.filter(category=category, tag=tag)
    
    # If no todos are found, return an empty list
    if not todos.exists():
        return JsonResponse({'message': 'No Todo items found'}, status=404)
    
    # Build a list of todos to return as a JSON response
    todos_list = list(todos.values('id', 'task', 'completed', 'category', 'tag'))
    
    return JsonResponse(todos_list, safe=False)
```

### Key Details:
1. **Input validation**: The code checks if both `category` and `tag` are provided using `request.GET.get('category')` and `request.GET.get('tag')`. If either is missing, a `400 Bad Request` is returned.
2. **Filtering the `Todo` items**: `Todo.objects.filter(category=category, tag=tag)` retrieves all `Todo` items that match the given `category` and `tag`.
3. **Returning JSON response**: The matching `Todo` items are serialized into a list of dictionaries using `list(todos.values('id', 'task', 'completed', 'category', 'tag'))`, which is then returned as a JSON response. `safe=False` is used to allow lists to be returned directly by `JsonResponse`.

### Testing the Endpoint:

- **Preloading the database**: This loads the initial set of `Todo` items with various categories and tags.

```python
import requests

URL = 'http://127.0.0.1:3000'

# Preload the database
response = requests.get(URL + '/preload/')
print(response.json())

# Valid request
response = requests.get(URL + '/todos/filter/?category=Home&tag=Urgent')
print(response.json())  # This should return the "Do the laundry" task

# Invalid request
response = requests.get(URL + '/todos/filter/?category=Invalid&tag=Invalid')
print(response.json())  # This should return a 404 with "No Todo items found"
```

### Sample Responses:

1. **Valid Request**:
    ```json
    [
        {
            "id": 1,
            "task": "Do the laundry",
            "completed": false,
            "category": "Home",
            "tag": "Urgent"
        }
    ]
    ```

2. **Invalid Request** (category and tag not found):
    ```json
    {
        "message": "No Todo items found"
    }
    ```

3. **Invalid Request** (missing category or tag):
    ```json
    {
        "message": "Category and/or Tag not provided"
    }
    ```

### URL Configuration:
You don't need to change the URLs, as the route already points to `get_todos_by_category_and_tag`:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('preload/', views.preload, name='preload'),
    path('todos/filter/', views.get_todos_by_category_and_tag, name='get_todos_by_category_and_tag'),
]
```

This should work well for filtering `Todo` items based on `category` and `tag`. Let me know if you need further tweaks!